In [193]:
from __future__ import print_function

import numpy as np

from keras import backend as K
from keras.models import Model
from keras.layers import Embedding, Flatten, Input, merge
from keras.optimizers import Adam

from keras.utils import plot_model
from keras.layers import Input,Layer,Lambda
from keras.layers import Flatten,BatchNormalization
from keras.layers import Dense,Dropout
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPooling2D
from keras import backend as K
# import data
# import metrics

誤差関数の定義

In [208]:
def triplet_loss(inputs):
    a, p, n = inputs
    p_dist = K.sum(K.square(a-p), axis=-1)
    n_dist = K.sum(K.square(a-n), axis=-1)
    return 1.0 - K.sum(K.maximum(p_dist - n_dist + alpha, 0), axis=0)
#     return K.sum(K.maximum(p_dist - n_dist + self.alpha, 0), axis=0)

In [210]:
imheight = 128
imwidth = 128
channels = 3
# ALPHA = 0.2
alpha=0.05

In [8]:
from keras.applications.vgg16 import VGG16
#include_top=false => Dense不要
base_model = VGG16(include_top=False, weights='imagenet', input_tensor=Input(shape=(imwidth, imheight, channels)), input_shape=None) 

15層目までは訓練しない

In [9]:
for layer in base_model.layers[:15]:
    layer.trainable=False

ネットワークを定義<br>

In [10]:
def create_embNet():
    x = base_model.output
    conv1 = Conv2D(32, (4,4) , padding='same', activation='relu')(x)
    conv2 = Conv2D(32, (4,4) , padding='same', activation='relu')(conv1)
    flatten = Flatten()(conv2) 
    dense_layer = Dense(20, activation='relu')(flatten)
    norm_layer = Lambda(lambda  x: K.l2_normalize(x, axis=1), name='norm_layer')(dense_layer)
    return  Model(inputs=[base_model.input], outputs=norm_layer)

In [217]:
def identity_loss(y_true, y_pred):

    return K.mean(y_pred - 0 * y_true)

In [218]:
def build_model():
    a_in = Input(shape = (imheight, imwidth, channels), name='anchor_input')
    p_in = Input(shape = (imheight, imwidth, channels), name='positive_input')
    n_in = Input(shape = (imheight, imwidth, channels), name='negative_input')
    
    a_emb = create_embNet()(a_in)
    p_emb = create_embNet()(p_in)
    n_emb = create_embNet()(n_in)
    
#     loss = merge(
#         [a_emb,p_emb, n_emb],
#         mode=triplet_loss,
#         name='loss',
#         output_shape=(1, ))
    
    loss = Lambda(lambda x: triplet_loss(x), output_shape=(1,))( [a_emb,p_emb, n_emb])
    
    model = Model(
        input=[a_in,p_in,n_in],
        output=loss)
    
#     model.compile(loss=identity_loss, optimizer=Adam())
    model.compile(loss=identity_loss, optimizer=Adam())
    
    return model

In [219]:
model = build_model()

/home/inouelab/.conda/envs/zukapy35/lib/python3.5/site-packages/ipykernel_launcher.py:20: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("la..., inputs=[<tf.Tenso...)`


元々の
- 'crop_imgs/img/CLOTHING/T_Shirt'
- 'crop_imgs/img/TOPS/T_Shirt' <br>
を結合したので
- './T_Shirt_all/'<br>
のみを参照すれば良い

## './T_Shirt_all/train/'と'./T_Shirt_all/test/'に分けたい

In [162]:
import os

BASE_PATH = './T_Shirt_all/'
ids = sorted([x for x in os.listdir(BASE_PATH)])

In [61]:
ids[0:5]

['id_00000001', 'id_00000019', 'id_00000051', 'id_00000058', 'id_00000068']

In [76]:
len(set(ids))

6377

In [77]:
def calc_filenums(ids):
    sum_ = 0
    for id_ in ids:
        files = sorted([BASE_PATH+id_+'/'+x for x in os.listdir(BASE_PATH+id_)])
        sum_ += len(files)
    return sum_

In [78]:
sum_ = calc_filenums(ids)
sum_

40115

In [20]:
def dirname2int(dirname):
    dirname = dirname[3::]
    return int(dirname)

### =======Warning!! 以下実行注意=======

In [160]:
import shutil
def rmdir(ids,PATH):
    for id_ in ids:
        files = sorted([BASE_PATH+id_+'/'+x for x in os.listdir(BASE_PATH+id_)])
        con = sorted([x for x in files if 'comsumer' in x])
        shop = sorted([x for x in files if 'shop' in x ])
        if len(con)==0 or len(shop)==0:
            shutil.rmtree(PATH+id_)    
    return os.listdir(BASE_PATH)

In [178]:
# ids = rmdir(ids,PATH=BASE_PATH)

### =================================

In [175]:
import itertools
import random
def get_triplets(ids,BASE_PATH):
    triplets=[]
    # idの選択の仕方をランダムにしたらいいのか？
    for id_ in ids:
        files = sorted([BASE_PATH+id_+'/'+x for x in os.listdir(BASE_PATH+id_)])
#         print(files)
        con = sorted([x for x in files if 'comsumer' in x])
        shop = sorted([x for x in files if 'shop' in x ])
        combs = list(itertools.product(tuple(con),tuple(shop)))
        
        for comb in combs:
#             print(len(comb))
            comb = list(comb)
            neg_id = random.choice([x for x in ids if x != id_])
#             print(neg_id)
#             print(len([BASE_PATH+neg_id+'/'+x for x in os.listdir(BASE_PATH+neg_id) if 'shop' in x]))
            neg_file = random.choice([BASE_PATH+neg_id+'/'+x for x in os.listdir(BASE_PATH+neg_id) if 'shop' in x])
            comb.append(neg_file)
            triplets.append(comb)
    
    return triplets

In [163]:
def sample_train_ids(ids):
    np.random.seed(seed=0)
    train_ids = np.random.choice(ids,int(0.75*len(ids)))
    return train_ids

In [167]:
int(0.75*len(ids))

4616

In [165]:
train_ids = sample_train_ids(ids)
#idsの中でtrain_idsに含まれていないxをtest_idsとする
test_ids = [x for x in ids if x not in train_ids] 

In [126]:
a = [dirname2int(x) for x in train_ids]
b = [dirname2int(x) for x in test_ids]

In [127]:
aaa = [3,5,6,7,9]
bbb = [1,2,3,4,5,6,7,8,9]
ccc = [x for x in bbb if x not in aaa]
print(ccc)

[1, 2, 4, 8]


In [128]:
# set(train_ids[:10])

In [129]:
# set(test_ids[:10])

In [166]:
print("train:{0}\ntest:{1}".format(len(train_ids),len(test_ids)))

train:4616
test:2906


In [168]:
l1_l2_and = set(a) & set(b)
print(l1_l2_and)

set()


合計が`len(ids)`とずれるのはおかしい

In [169]:
len(train_ids)+len(test_ids)

7522

In [179]:
print("train_images:{}\ntest_images:{}".format(calc_filenums(train_ids),calc_filenums(test_ids)))

train_images:29918
test_images:18774


In [171]:
len(train_ids)

4616

In [152]:
os.listdir(BASE_PATH+'id_00031391')

['comsumer_01.jpg']

In [181]:
x_train = get_triplets(train_ids,BASE_PATH) 
x_train[:5]

[['./T_Shirt_all/id_00015437/comsumer_01.jpg',
  './T_Shirt_all/id_00015437/shop_01.jpg',
  './T_Shirt_all/id_00008148/shop_01.jpg'],
 ['./T_Shirt_all/id_00014893/comsumer_01.jpg',
  './T_Shirt_all/id_00014893/shop_01.jpg',
  './T_Shirt_all/id_00011992/shop_01.jpg'],
 ['./T_Shirt_all/id_00014893/comsumer_01.jpg',
  './T_Shirt_all/id_00014893/shop_02.jpg',
  './T_Shirt_all/id_00000760/shop_01.jpg'],
 ['./T_Shirt_all/id_00014893/comsumer_02.jpg',
  './T_Shirt_all/id_00014893/shop_01.jpg',
  './T_Shirt_all/id_00008383/shop_01.jpg'],
 ['./T_Shirt_all/id_00014893/comsumer_02.jpg',
  './T_Shirt_all/id_00014893/shop_02.jpg',
  './T_Shirt_all/id_00015025/shop_01.jpg']]

In [190]:
from PIL import Image
def get_np_triplets(triplet_PATHs):
    triplets = []
    # triplets = np.ndarray
    for triplet in triplet_PATHs:

#         anc_img = Image.fromarray(np.uint8(triplet[0])).convert('RGB')
#         pos_img = Image.fromarray(np.uint8(triplet[1])).convert('RGB')
#         neg_img = Image.fromarray(np.uint8(triplet[2])).convert('RGB')

        anc_img = Image.open(triplet[0]).convert('RGB')
        pos_img = Image.open(triplet[1]).convert('RGB')
        neg_img = Image.open(triplet[2]).convert('RGB')

        anc_img = np.array(anc_img.resize((128,128)))/255. #resize to (128,128,3)
        pos_img = np.array(pos_img.resize((128,128)))/255.    
        neg_img = np.array(neg_img.resize((128,128)))/255.    

        tri = [anc_img,pos_img,neg_img]
        triplets.append(np.array(tri))

    triplets = np.array(triplets)
    return triplets

### 1epochのfitをfor文で回す
これによりnb_epochで指定するよりもコードに自由度が出る
- 1epochごとにtrainのアイテム集合からtripletを作成する。
- 毎回nidが変わるので乱数によるnidのばらつきを吸収できる。

In [188]:
num_epochs = 10

In [200]:
model = build_model()

TypeError: ('Keyword argument not understood:', 'mode')

In [220]:
for epoch in range(num_epochs):

    print('Epoch %s' % epoch)

    # Sample triplets from the training data
    x_train_PATHs = get_triplets(train_ids,BASE_PATH)
    X = get_np_triplets(x_train_PATHs)
    
#     X = {
#         'anchor_input': uid,
#         'positive_input': pid,
#         'negative_input': nid
#     }

    model.fit(X,
              np.ones(len(X)),
              batch_size=64,
              nb_epoch=1,
              verbose=1,
              shuffle=True)

#     print('AUC %s' % metrics.full_auc(model, test))


Epoch 0


/home/inouelab/.conda/envs/zukapy35/lib/python3.5/site-packages/ipykernel_launcher.py:20: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


ValueError: Error when checking model input: the list of Numpy arrays that you are passing to your model is not the size the model expected. Expected to see 3 array(s), but instead got the following list of 1 arrays: [array([[[[[0.79215686, 0.5372549 , 0.38039216],
          [0.79215686, 0.5372549 , 0.38039216],
          [0.74901961, 0.49411765, 0.3372549 ],
          ...,
          [0.13333333, 0.1254902 , 0.145...

In [ ]:
for epoch in range(num_epochs):

    print('Epoch %s' % epoch)

    # Sample triplets from the training data
    uid, pid, nid = get_triplets(train)
    

    X = {
        'user_input': uid,
        'positive_item_input': pid,
        'negative_item_input': nid
    }

    model.fit(X,
              np.ones(len(uid)),
              batch_size=64,
              nb_epoch=1,
              verbose=0,
              shuffle=True)

    print('AUC %s' % metrics.full_auc(model, test))